In [1]:
import kmodels as kmk
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
from torch.nn import functional as F
import pickle
import gc

c:\Users\efons\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_pickle(r'C:\Users\efons\Desktop\kalyn_lstm\kmk\data\SeparatedData_MedFilt_v2-Copy1.pkl')

In [3]:
leave_out_df = df[df['Subject'] == 'RGr10']
train_df = df[df['Subject'] != 'RGr10']
window_size = 100
stride = window_size
train_data = kmk.Dataset_LSTM_experimental(train_df, window_size=window_size, stride=stride)
test_data = kmk.Dataset_LSTM_experimental(leave_out_df, window_size=window_size, stride=stride)
train_data.xcols

['EPB', 'EPL', 'FPL', 'APL', 'ADD', 'FCU', 'FPB', 'OPP']

In [4]:
xcols = train_data.xcols
xcols.append('Time')
train_data = kmk.Dataset_LSTM_experimental(train_df, window_size=window_size, stride=stride, xcols=xcols)
test_data = kmk.Dataset_LSTM_experimental(leave_out_df, window_size=window_size, stride=stride, xcols=xcols)

In [5]:
train_data.xcols, train_data.X.shape

(['EPB', 'EPL', 'FPL', 'APL', 'ADD', 'FCU', 'FPB', 'OPP', 'Time'],
 torch.Size([42, 100, 9]))

In [ ]:
models = kmk.random_models_LSTM(25, batch_size_range=(2, len(train_data)), n_linear_layers_range=(2, 4))
models

In [ ]:
results = kmk.train_LSTMs(models, train_data, train_data, n_epochs=200)

In [ ]:
plt.bar(range(len(results[3])), results[3]['mae'])
# put the mae on the bars
for i, v in enumerate(results[3]['mae']):
    plt.text(i, v, str(round(v, 2)), color='blue', fontweight='bold')
    

In [ ]:
sim_df = pd.read_pickle(r'C:\Users\efons\Desktop\kalyn_lstm\kmk\data\LP_Simulations_Formatted.pkl')
window_size = 100
stride = window_size
sim_df.columns

In [ ]:
# first lets split the dataframe into a train and test set using the Subject column. Keep subjects separate during training and testing
#df = dd.from_pandas(df, npartitions=4)
subject_dfs = kmk.split_df_into_subjects(sim_df)
train_split = 0.9
n_train_subjects = int(len(subject_dfs) * train_split)
n_test_subjects = len(subject_dfs) - n_train_subjects


In [ ]:
# randomly pick the subjects to use for training and testing
train_subjects_index = np.random.choice(len(subject_dfs), n_train_subjects, replace=False)
train_df_sim = pd.concat([subject_dfs[i] for i in train_subjects_index])
test_df_sim = pd.concat([subject_dfs[i] for i in range(len(subject_dfs)) if i not in train_subjects_index])


In [ ]:
train_data_sim = kmk.Dataset_LSTM_opensim(train_df_sim, sort_column=['Subject'], window_size=window_size, stride=stride)

In [ ]:
test_data_sim = kmk.Dataset_LSTM_opensim(test_df_sim, sort_column=['Subject'], window_size=window_size, stride=stride)

In [ ]:
train_data_sim.X = torch.from_numpy(train_data_sim.X).float()
train_data_sim.Y = torch.from_numpy(train_data_sim.Y).float()
test_data_sim.X = torch.from_numpy(test_data_sim.X).float()
test_data_sim.Y = torch.from_numpy(test_data_sim.Y).float()

In [ ]:
train_data_sim

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,10))
ax[0].plot(train_data[1][0])
ax[0].set_xlabel('time steps')
ax[0].set_ylabel('Muscle Activation')
ax[1].plot(train_data[1][1])
ax[1].set_xlabel('time steps')
ax[1].set_ylabel('Force')

In [ ]:
#model = lstm(n_linear_layers=3, n_outputs = len(train_data.Y[0].flatten()), lstm_n_outputs = 3000, layer_size=30).to('cuda')
best_model_index = np.argmin(results[3]['mae'])
model = results[0]
learning_rate = results[2]['learning_rate'][best_model_index]
batch_size = results[2]['batch_size'][best_model_index]
batch_size = 224
batch_size

In [ ]:
losses = kmk.run_Pytorch(model, train_data_sim.X.reshape(-1, 100, 8), train_data_sim.Y, n_epochs=101, batch_size=batch_size, learning_rate=learning_rate, device='cuda')

In [ ]:
def TLLSTM(model, X, change_layers=1):
    new = kmk.lstm(n_linear_layers=model.n_linear_layers, n_outputs=model.output.out_features, lstm_n_outputs=model.lstm_n_outputs, layer_size=model.layer_size, hidden_size=36)
    test = new(X)
    new.load_state_dict(model.state_dict())
    children = [child for child in new.children()]
    for child in children:
        for param in child.parameters():
            param.requires_grad = False
    total_layers = len(children)
    for i in range(change_layers):
        layer = children[total_layers-i-1]
        layer_params = layer.parameters()
        for p in layer_params:
            p.requires_grad = True
    return new

# makes make another TLMLSTM_newlayers function
def TLMLSTM_newlayers(model, X, change_layers=1, new_layer_size=10):
    new = kmk.lstm(n_linear_layers=model.n_linear_layers, 
               n_outputs=model.output.out_features, 
               lstm_n_outputs=model.lstm_n_outputs, 
               layer_size=model.layer_size)
    test = new(X)
    new.load_state_dict(model.state_dict())
    children = [child for child in new.children()]
    for child in children:
        for param in child.parameters():
            param.requires_grad = False
    total_layers = len(children)
    # add the new layers
    new.new_layer_size = new_layer_size
    new.layers = len(new.fcs) + change_layers
    for i in range(change_layers):
        if i == 0:
            new.fcs.append(nn.Linear(new.layer_size, new.new_layer_size))
        else:
            new.fcs.append(nn.Linear(new.new_layer_size, new.new_layer_size))
            # make sure the new layers are trainable
        new.fcs[-1].requires_grad = True
    # make the new output layer 
    new.output = nn.Linear(new.new_layer_size, new.output.out_features)
    return new

In [ ]:
tlm = TLLSTM(model, train_data_sim.X[0].reshape(1, 100, 8), change_layers=3)
tlm

In [ ]:
loss_tlm = kmk.run_Pytorch(tlm, train_data.X, train_data.Y, n_epochs=2001, batch_size=results[2].iloc[best_model_index]['batch_size'], learning_rate=learning_rate, device='cuda')

In [ ]:
# now to plot a paper worthy learning curve of the model. Make the line stand out in a clean cartoonish way. Make the y axis
# log scale. 
def plot_learning_curve(losses, title='Learning Curve', xlabel='Epoch', ylabel='Loss', figsize=(10,10)):
    fig, ax = plt.subplots(1,1, figsize=figsize)
    ax.plot(losses)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yscale('log')
    # add a grid
    ax.grid()
    return fig, ax
plot_learning_curve(loss_tlm, title='Learning Curve', xlabel='Epoch', ylabel='Loss', figsize=(10,10))

In [ ]:
figname = 'FFNN_FineTuned_ExpTest.png'
import math
# Functions
def roundup(x, nearest = 5):
    return int(math.ceil(x / nearest)) * nearest

def rounddown(x, nearest = 5):
    return int(math.floor(x / nearest)) * nearest

# Parity subplots
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
Y = test_data.Y.reshape(-1,3).detach().cpu().numpy()
pred = model(test_data.X.reshape(-1, 100, 8)).detach().cpu().numpy().reshape(-1,3)
X_true = Y[:,0]
X_pred = pred[:,0]
Y_true = Y[:,1]
Y_pred = pred[:,1]
Z_true = Y[:,2]
Z_pred = pred[:,2]

# Plot predictions of X 
ax1.scatter(X_true, X_pred, s=50, alpha=0.5, edgecolors='k', color='red')
ax1.set_title('Distal (+)', fontsize = 20)
ax1.set_xlabel('True Force (N)', fontsize = 20)
ax1.set_ylabel('Predicted Force (N)', fontsize = 20)
ax1.set_aspect('equal')

# Set min and max range for axes
alldata = np.concatenate((np.array(X_true), X_pred), axis=0)
max_range = roundup(alldata.max())
min_range = rounddown(alldata.min())
ax1.set_xlim(min_range, max_range)
ax1.set_ylim(min_range, max_range)
ax1.set_xticks([min_range, max_range])
ax1.set_yticks([min_range, max_range])
ax1.tick_params(axis='both', which = 'major', labelsize = 16)

ax1.plot([0, 1], [0, 1], '--k', linewidth=5, transform=ax1.transAxes)



# Plot predictions of Y
ax2.scatter(Y_true, Y_pred, s=50, alpha=0.5, edgecolors='k', color = 'green')
ax2.set_title('Dorsal (+)', fontsize = 20)
ax2.set_xlabel('True Force (N)', fontsize = 20)
ax2.set_aspect('equal')

# Set min and max range for axes
alldata = np.concatenate((np.array(Y_true), Y_pred), axis=0)
max_range = roundup(alldata.max())
min_range = rounddown(alldata.min())
ax2.set_xlim(min_range, max_range)
ax2.set_ylim(min_range, max_range)
ax2.set_xticks([min_range, max_range])
ax2.set_yticks([min_range, max_range])
ax2.tick_params(axis='both', which = 'major', labelsize = 16)

ax2.plot([0, 1], [0, 1], '--k', linewidth=5, transform=ax2.transAxes)




# Plot predictions of Z
ax3.scatter(Z_true, Z_pred, s=50, alpha=0.5, edgecolors='k', color = 'blue')
ax3.set_title('Ulnar (+)', fontsize = 20)
ax3.set_xlabel('True Force (N)', fontsize = 20)
ax3.set_aspect('equal')

# Set min and max range for axes
alldata = np.concatenate((np.array(Z_true), Z_pred), axis=0)
max_range = roundup(alldata.max())
min_range = rounddown(alldata.min())
ax3.set_xlim(min_range, max_range)
ax3.set_ylim(min_range, max_range)
ax3.set_xticks([min_range, max_range])
ax3.set_yticks([min_range, max_range])
ax3.tick_params(axis='both', which = 'major', labelsize = 16)

ax3.plot([0, 1], [0, 1], '--k', linewidth=5, transform=ax3.transAxes)
plt.savefig(figname, dpi = 600)

In [ ]:
# plot a sample of the data

# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]

n = np.random.randint(0, len(test_data.X))
print(len(test_data.X))
x_plot = test_data.X.reshape(-1, 8).float().detach().cpu().numpy()
y_plot = test_data.Y.reshape(-1, 3).float().detach().cpu().numpy()
dt = test_data.df['Time'].iloc[1] - test_data.df['Time'].iloc[0]
time = np.arange(0, x_plot.shape[0]*dt, dt)
print(y_plot.shape)
y_labels = test_data.ycols
x_labels = test_data.xcols
# lets remake this plot but make the two subplots share the x axis
fig, ax = plt.subplots(2, 1, figsize=(10,8), sharex=True)  
# make sure the colors for xplot do not overlap with yplot. Make xplot from the viridis colormap. there are 8 columns in xplot
for i in range(x_plot.shape[1]):
    ax[0].plot(time, x_plot[:,i], color=plt.cm.tab20b(i/x_plot.shape[1]))
ax[0].set_ylabel('Muscle Activation')    
# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]
for i in range(y_plot.shape[1]):
    ax[1].plot(time, y_plot[:,i], color=line_colors[i])
ax[1].set_xlabel('Time [s]')
ax[1].set_ylabel('Force [N]')
ax[1].legend(y_labels)
#plot the prediction of this sample 
pred = tlm(test_data.X.to('cuda')).detach().cpu().numpy().reshape(-1, 3)
print(pred.shape)
for i in range(pred.shape[1]):
    ax[1].plot(time, pred[:,i], color=line_colors[i], linestyle='--')
# print the mean prediction on the plot
pred_mean = np.mean(pred, axis=0)
print(pred_mean)

In [ ]:
# plot a sample of the data

# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]

n = np.random.randint(0, len(test_data.X))
print(len(test_data.X))
x_plot = test_data.X.reshape(-1, 8).float().detach().cpu().numpy()
y_plot = test_data.Y.reshape(-1, 3).float().detach().cpu().numpy()
dt = test_data.df['Time'].iloc[1] - test_data.df['Time'].iloc[0]
time = np.arange(0, x_plot.shape[0]*dt, dt)
print(y_plot.shape)
y_labels = test_data.ycols
x_labels = test_data.xcols
# lets remake this plot but make the two subplots share the x axis
fig, ax = plt.subplots(2, 1, figsize=(10,8), sharex=True)  
# make sure the colors for xplot do not overlap with yplot. Make xplot from the viridis colormap. there are 8 columns in xplot
for i in range(x_plot.shape[1]):
    ax[0].plot(time, x_plot[:,i], color=plt.cm.tab20b(i/x_plot.shape[1]))
ax[0].set_ylabel('Muscle Activation')    
# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]
for i in range(y_plot.shape[1]):
    ax[1].plot(time, y_plot[:,i], color=line_colors[i])
ax[1].set_xlabel('Time [s]')
ax[1].set_ylabel('Force [N]')
ax[1].legend(y_labels)
#plot the prediction of this sample 
pred = tlm(test_data.X.to('cuda')).detach().cpu().numpy().reshape(-1, 3)
print(pred.shape)
for i in range(pred.shape[1]):
    ax[1].plot(time, pred[:,i], color=line_colors[i], linestyle='--')
# print the mean prediction on the plot
pred_mean = np.mean(pred, axis=0)
print(pred_mean)

In [ ]:
# plot a sample of the data

# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]

n = np.random.randint(0, len(train_data.X))
print(len(train_data.X))
x_plot = np.abs(train_data.X.reshape(-1, 8).float().detach().cpu().numpy())
y_plot = np.abs(train_data.Y.reshape(-1, 3).float().detach().cpu().numpy())
dt = train_data.df['Time'].iloc[1] - train_data.df['Time'].iloc[0]
time = np.arange(0, x_plot.shape[0]*dt, dt)
print(y_plot.shape)
y_labels = train_data.ycols
x_labels = train_data.xcols
# lets remake this plot but make the two subplots share the x axis
fig, ax = plt.subplots(2, 1, figsize=(15,8), sharex=True)  
# make sure the colors for xplot do not overlap with yplot. Make xplot from the viridis colormap. there are 8 columns in xplot
for i in range(x_plot.shape[1]):
    ax[0].plot(time, x_plot[:,i], color=plt.cm.tab20b(i/x_plot.shape[1]))
ax[0].set_ylabel('Muscle Activation')    
# line_colors = red, green, blue
line_colors = [(1,0,0), (0,1,0), (0,0,1)]
for i in range(y_plot.shape[1]):
    ax[1].plot(time, y_plot[:,i], color=line_colors[i])
ax[1].set_xlabel('Time [s]')
ax[1].set_ylabel('Force [N]')
ax[1].legend(y_labels)
#plot the prediction of this sample 
pred = np.abs(tlm(train_data.X.to('cuda')).detach().cpu().numpy().reshape(-1, 3))
print(pred.shape)
for i in range(pred.shape[1]):
    ax[1].plot(time, pred[:,i], color=line_colors[i], linestyle='--')
# print the mean prediction on the plot
pred_mean = np.mean(pred, axis=0)
print(pred_mean)
#ax[1].set_yscale('log')
ax[1].set_xlim([0, 20])

In [6]:
import time

def run_Pytorch(model, X_train, Y_train, n_epochs=100, learning_rate=1e-5, batch_size=int(1e5), device='cuda', optimizer=None):
    torch.cuda.empty_cache()
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
    losses = train_pytorch(model, 
                 X_train, 
                 Y_train,
                 n_epochs=n_epochs,
                 batch_size=batch_size, 
                 learning_rate=learning_rate)
    return losses

def run_epochs(model, X_train, Y_train, loss_func, optimizer, batches, n_epochs=100, device='cuda'):
    t1 = time.time()
    losses = []
    loss_func = loss_func.to(device)
    for epoch in range(n_epochs):
        for i in batches:
           # i = indicies[i]
            optimizer.zero_grad()   # clear gradients for next train
            x = X_train[i,:].to(device)
            y = Y_train[i,:].to(device).flatten()
            pred = model(x).flatten()
            # check if y and pred are the same shape
            # if y.shape != pred.shape:
            #     print('y and pred are not the same shape')
            #     print(y.shape, pred.shape)
            #     break
            # if x.device != y.device:
            #     print('x and y are not on the same device')
            #     print(x.device, y.device)
            #     break
            # if model.device != x.device:
            #     print('model and x are not on the same device')
            #    print(model.device, x.device)
            #    break
            loss = loss_func(pred, y) # must be (1. nn output, 2. target)
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
        losses.append(loss)
        torch.cuda.empty_cache()
        gc.collect()
        if epoch%10 == 0:
            t2 = time.time()
            print('EPOCH : ', epoch,', dt: ',
                  t2 - t1, 'seconds, losses :', 
                  float(loss.detach().cpu())) 
            t1 = time.time()
    return losses

def train_pytorch(model, X_train, Y_train, n_epochs=1000, batch_size=int(1e3), learning_rate=1e-3, device='cuda', optimizer=None):
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)
    losses = []
    batches = batch_data(X_train, batch_size)
    model = model.to(device)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.MSELoss()
    losses = run_epochs(model, X_train, Y_train, loss_func, optimizer, batches, n_epochs=n_epochs)
    return [i.detach().cpu() for i in losses]

def batch_data(Y, batch_size):
    batch_size = int(batch_size)
    n_observations = int(Y.shape[0])
    batch_index = np.arange(0, n_observations, batch_size)
    #np.random.shuffle(batch_index)
    batches = np.array([np.arange(batch_index[i], batch_index[i+1]) \
                   for i in range(len(batch_index)-1)])
    shape = batches.shape
    temp = batches.reshape(-1,1)
    np.random.shuffle(temp)
    batches = temp.reshape(shape[0], shape[1])
    np.random.shuffle(batches)
    n_batches = len(batches)
    return batches

In [7]:
class lstm(nn.Module):
    def __init__(self, n_inputs=8, hidden_size=30, n_outputs=300, n_linear_layers=1, 
                 layer_size=10, lstm_n_outputs=30):
        super().__init__()
        self.lstm = nn.LSTM(n_inputs, hidden_size, batch_first=True)
        self.fcs = nn.ModuleList([nn.Linear(layer_size, layer_size) for i in range(n_linear_layers)])
        self.layer_size = layer_size
        self.n_linear_layers = n_linear_layers 
        self.lstm_n_outputs = lstm_n_outputs
        self.output = nn.Linear(layer_size, n_outputs)
        self.device = torch.device('cpu')
        
    def forward(self, x):
        rows = x.shape[0]
        x, _ = self.lstm(x)
        x = x.reshape(rows, -1)
        for i, fc in enumerate(self.fcs):
            if fc == self.fcs[0]:
                if x.shape[1] != int(fc.in_features):
                    try:    
                        self.fcs[0] = nn.Linear(x.shape[1], self.layer_size)
                        x = F.relu(self.fcs[0](x))
                    except:
                        self.fcs[0] = nn.Linear(x.shape[1], self.layer_size).to(self.device)
                        x = F.relu(self.fcs[0](x))
                else:
                    x = F.relu(fc(x))
            else:
                x = F.relu(fc(x))
        x = self.output(x)
        return x
    
    def to(self, *args, **kwargs):
        super().to(*args, **kwargs)
        self.lstm = self.lstm.to(*args, **kwargs)
        for i, fc in enumerate(self.fcs):
            self.fcs[i] = fc.to(*args, **kwargs)
        self.output = self.output.to(*args, **kwargs)
        self.device =torch.device(*args, **kwargs)
        
        return self

In [8]:
class LSTM3(nn.Module):
    def __init__(self, input_size = 8, hidden_size = 128, output_size = 100, num_layers = 4, layer_size = 128):
        super(LSTM3, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.output_size = output_size
        self.layer_size = layer_size
        # we will use 3 lstms from kmk.lstm 
        self.lstms = nn.ModuleList([lstm(n_inputs=input_size, hidden_size=hidden_size, n_outputs=output_size,
                                         layer_size=layer_size) for i in range(3)])
        self.device = 'cpu'
        # x, y, z forces being predicted by each model independantly
    def forward(self, x):
        # x is a tensor of shape (batch_size, seq_len, input_size)
        # we will run each lstm on the same input
        prediction = torch.zeros((x.shape[0], x.shape[1], 3)).to(self.device)
        for i in range(3):
            prediction[:, :, i] = self.lstms[i](x)
        prediction = prediction.reshape(-1, 3*x.shape[1])
        return prediction
    # create a new "to" method to ensure all the models stored on this model get transferred to the same device
    def to(self, device):
        super().to(device)
        for i in range(3):
            self.lstms[i].to(device)
            self.lstms.device = device
        self.device = torch.device(device)
        return self
    
    def n_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [9]:
l = LSTM3(input_size=9, layer_size=2, num_layers=5)
l = l.to('cuda:0')
#l = lstm(n_inputs=8, layer_size=2, n_outputs=300, n_linear_layers=20)
#train_data.X.shape[0]*train_data.X.shape[1], n_params(l)

In [10]:
X = train_data.X.to('cuda:0')
Y = train_data.Y.to('cuda:0')
print(l)
lr = 1e-3
loss = run_Pytorch(l, X, Y,  learning_rate=lr,
                    n_epochs=1001, batch_size=4, device=torch.device('cuda:0'), optimizer=torch.optim.Adam(l.parameters(), lr=lr, weight_decay=0.01))



LSTM3(
  (lstms): ModuleList(
    (0): lstm(
      (lstm): LSTM(9, 128, batch_first=True)
      (fcs): ModuleList(
        (0): Linear(in_features=2, out_features=2, bias=True)
      )
      (output): Linear(in_features=2, out_features=100, bias=True)
    )
    (1): lstm(
      (lstm): LSTM(9, 128, batch_first=True)
      (fcs): ModuleList(
        (0): Linear(in_features=2, out_features=2, bias=True)
      )
      (output): Linear(in_features=2, out_features=100, bias=True)
    )
    (2): lstm(
      (lstm): LSTM(9, 128, batch_first=True)
      (fcs): ModuleList(
        (0): Linear(in_features=2, out_features=2, bias=True)
      )
      (output): Linear(in_features=2, out_features=100, bias=True)
    )
  )
)
EPOCH :  0 , dt:  1.904541254043579 seconds, losses : 1108.147216796875
EPOCH :  10 , dt:  3.4705891609191895 seconds, losses : 1064.943115234375
EPOCH :  20 , dt:  3.4917709827423096 seconds, losses : 1014.2593994140625
EPOCH :  30 , dt:  3.411687135696411 seconds, losses : 949.

: 

In [ ]:
# now to plot a paper worthy learning curve of the model. Make the line stand out in a clean cartoonish way. Make the y axis
# log scale. 
def plot_learning_curve(losses, title='Learning Curve', xlabel='Epoch', ylabel='Loss', figsize=(10,10)):
    fig, ax = plt.subplots(1,1, figsize=figsize)
    ax.plot(losses)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yscale('log')
    # add a grid
    ax.grid()
    return fig, ax
plot_learning_curve(loss, title='Learning Curve', xlabel='Epoch', ylabel='Loss', figsize=(10,10))

In [ ]:
X = test_data.X.to('cuda:0')
Y = np.abs(test_data.Y.to('cpu').reshape(-1, 3)).cpu().numpy()
pred = np.abs(l(X).reshape(-1, 3).detach().cpu().numpy())

Force_colors = ['r', 'g', 'b']
for i in range(3):
    plt.plot(Y[:, i], Force_colors[i], label=f'Y_{i}')


for i in range(3):
    plt.plot(pred[:, i], Force_colors[i], label=f'pred_{i}', linestyle='--')

plt.yscale('log')
#pred.shape, Y.shape, Fx.shape, Fy.shape

In [ ]:
# parity plot
plt.scatter(Y[:, 0], pred[:, 0], color=Force_colors[0], label='Fx', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.scatter(Y[:, 1], pred[:, 1], color=Force_colors[1], label='Fy', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.scatter(Y[:, 2], pred[:, 2], color=Force_colors[2], label='Fz', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.legend()
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, color='k', linestyle='--')

In [ ]:
# plot magnitude of forces as a parity plot
F = np.sqrt(Y[:, 0]**2 + Y[:, 1]**2 + Y[:, 2]**2)
F_pred = np.sqrt(pred[:, 0]**2 + pred[:, 1]**2 + pred[:, 2]**2)
plt.scatter(F, F_pred, color='k', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, color='k', linestyle='--')

In [ ]:
x_linear = train_data.X.reshape(-1, 8).to('cpu').numpy()
y_linear = train_data.Y.reshape(-1, 3).to('cpu').numpy()[:,1]

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
# reg   = LinearRegression().fit(x_linear, y_linear)
# reg = RandomForestRegressor(n_estimators = 100, max_depth=10, random_state=0).fit(x_linear, y_linear)
reg = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=0).fit(x_linear, y_linear)
reg.fit(x_linear, y_linear)

In [ ]:
pred = reg.predict(test_data.X.reshape(-1, 8).to('cpu').numpy())
y_plot = test_data.Y.reshape(-1, 3).to('cpu').numpy()[:,1]
plt.scatter(y_plot, pred, color=Force_colors[0], label='Fx', edgecolors='k', linewidths=0.5, alpha=0.5)  
# plt.scatter(y_plot[:,1], pred[:, 1], color=Force_colors[1], label='Fy', edgecolors='k', linewidths=0.5, alpha=0.5)
# plt.scatter(y_plot[:,2], pred[:, 2], color=Force_colors[2], label='Fz', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, color='k', linestyle='--')
mae = np.mean(np.abs(y_plot - pred))
mse = np.mean((y_plot - pred)**2)
r2 = 1 - np.sum((y_plot - pred)**2)/np.sum((y_plot - np.mean(y_plot))**2)
print('MAE : ', mae)
print('MSE : ', mse)
print('R2 : ', r2)
mae

In [ ]:
n_estimators_range = [10, 50, 100, 200, 500, 1000]
n_depths = [5, 10, 20, 50, 100]
leaf_splits = [2, 5, 10, 20, 50, 100]
from sklearn.model_selection import RandomizedSearchCV
cv_search = RandomizedSearchCV(RandomForestRegressor(), param_distributions={'n_estimators': n_estimators_range, 'max_depth': n_depths, 'min_samples_leaf': leaf_splits}, n_iter=10, n_jobs=-1, cv=5, verbose=3)
cv_search.fit(x_linear, y_linear)

In [ ]:
cv_search.best_score_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=0).fit(x_linear, y_linear)
reg.fit(x_linear, y_linear)
mae = np.mean(np.abs(y_plot - pred))
mae

In [ ]:
# get SVR from sklearn
from sklearn.svm import SVR
reg = SVR().fit(x_linear, y_linear)
pred = reg.predict(test_data.X.reshape(-1, 8).to('cpu').numpy())
plt.scatter(y_plot, pred, color=Force_colors[0], label='Fx', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, color='k', linestyle='--')
mae = np.mean(np.abs(y_plot - pred))
mae

In [ ]:
x_linear.shape

In [ ]:
# get the gaussian process regressor from sklearn
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(hidden_layer_sizes=(8, 8, 8)).fit(x_linear, y_linear)
pred = reg.predict(test_data.X.reshape(-1, 8).to('cpu').numpy())
plt.scatter(y_plot, pred, color=Force_colors[0], label='Fx', edgecolors='k', linewidths=0.5, alpha=0.5)
plt.plot([0, 1], [0, 1], transform=plt.gca().transAxes, color='k', linestyle='--')
mae = np.mean(np.abs(y_plot - pred))
mae

In [ ]:
# create a function that given a pytorch model will return the total number of learnable parameters

def n_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


l = LSTM3(input_size=8, output_size=100, layer_size=1, num_layers=1)

len(l.state_dict())
